In [0]:
# Notebook de Teste: Verificação do Ambiente Databricks para Projeto MLSecOps
# Autor: Luana Gonçalves
# Data: 5 de abril de 2025

# 1. Verificação de versões de bibliotecas essenciais
print("=== Verificando versões de bibliotecas essenciais ===")

import sys
print(f"Python version: {sys.version}")

# Verificando PySpark
try:
    import pyspark
    print(f"PySpark version: {pyspark.__version__}")
except ImportError:
    print("⚠️ PySpark não está disponível")

# Verificando Delta Lake
try:
    from delta import *
    print(f"Delta Lake está disponível")
except ImportError:
    print("⚠️ Delta Lake não está disponível")

# Verificando bibliotecas para ML
try:
    import pandas as pd
    import numpy as np
    print(f"Pandas version: {pd.__version__}")
    print(f"NumPy version: {np.__version__}")
except ImportError as e:
    print(f"⚠️ Erro ao importar bibliotecas de análise de dados: {e}")

try:
    import sklearn
    print(f"Scikit-learn version: {sklearn.__version__}")
except ImportError:
    print("⚠️ Scikit-learn não está disponível")

try:
    import mlflow
    print(f"MLflow version: {mlflow.__version__}")
except ImportError:
    print("⚠️ MLflow não está disponível")

# 2. Teste de conexão com Spark
print("\n=== Testando conexão com Spark ===")
try:
    # Criando um DataFrame de teste
    data = [("1", "Transação Normal", 500.0, False),
            ("2", "Transação Suspeita", 9999.0, True)]
    
    columns = ["id", "tipo", "valor", "fraude"]
    
    df = spark.createDataFrame(data, columns)
    print("DataFrame de teste criado com sucesso:")
    df.show()
    
    # Testando algumas operações Spark
    print("\nOperações básicas com Spark:")
    df_filtrado = df.filter(df.fraude == True)
    print("Filtrando transações fraudulentas:")
    df_filtrado.show()
    
    # Testando SQL
    print("\nRegistrando como tabela temporária e executando SQL:")
    df.createOrReplaceTempView("transacoes_teste")
    spark.sql("SELECT * FROM transacoes_teste WHERE valor > 1000").show()
    
except Exception as e:
    print(f"⚠️ Erro na conexão com Spark: {e}")

# 3. Teste de escrita e leitura com Delta Lake
print("\n=== Testando Delta Lake ===")
try:
    # Caminho temporário para teste
    temp_path = "/tmp/delta_test"
    
    # Salvando o DataFrame como tabela Delta
    print(f"Salvando dados como Delta no caminho: {temp_path}")
    df.write.format("delta").mode("overwrite").save(temp_path)
    
    # Lendo dados do Delta
    print("Lendo dados da tabela Delta:")
    delta_df = spark.read.format("delta").load(temp_path)
    delta_df.show()
    
    # Testando operações do Delta Lake
    print("\nVerificando histórico de versões da tabela Delta:")
    delta_table = DeltaTable.forPath(spark, temp_path)
    delta_table.history().show(5, truncate=False)
    
    print("\nDelta Lake está funcionando corretamente!")
except Exception as e:
    print(f"⚠️ Erro ao testar Delta Lake: {e}")

# 4. Teste de criação de banco de dados e tabelas para a arquitetura Medallion
print("\n=== Testando criação de bancos de dados para arquitetura Medallion ===")
try:
    # Criando os bancos de dados
    spark.sql("CREATE DATABASE IF NOT EXISTS bronze")
    spark.sql("CREATE DATABASE IF NOT EXISTS silver")
    spark.sql("CREATE DATABASE IF NOT EXISTS gold")
    
    # Verificando se os bancos foram criados
    print("Bancos de dados disponíveis:")
    spark.sql("SHOW DATABASES").show()
    
    # Criando uma tabela de teste no banco bronze
    print("\nCriando tabela de teste na camada Bronze:")
    df.write.format("delta").mode("overwrite").saveAsTable("bronze.transacoes_teste")
    
    # Verificando a tabela criada
    print("Tabelas no banco de dados bronze:")
    spark.sql("SHOW TABLES IN bronze").show()
    
    # Lendo a tabela
    print("Lendo dados da tabela bronze.transacoes_teste:")
    spark.sql("SELECT * FROM bronze.transacoes_teste").show()
    
except Exception as e:
    print(f"⚠️ Erro ao testar criação de bancos de dados: {e}")

# 5. Informações sobre o ambiente Spark
print("\n=== Informações sobre o ambiente Spark ===")
try:
    # Configurações do Spark
    print("Configurações do Spark:")
    for config in ["spark.app.name", "spark.driver.memory", 
                  "spark.executor.memory", "spark.executor.cores"]:
        try:
            value = spark.conf.get(config)
            print(f"  {config}: {value}")
        except:
            print(f"  {config}: Não disponível")
    
    # Informações sobre o cluster
    print("\nInformações sobre o ambiente:")
    sc_conf = spark.sparkContext.getConf()
    master = sc_conf.get("spark.master")
    app_id = sc_conf.get("spark.app.id")
    
    print(f"  Spark Master: {master}")
    print(f"  Application ID: {app_id}")
    
except Exception as e:
    print(f"⚠️ Erro ao obter informações do ambiente: {e}")

print("\n=== Verificação de ambiente concluída ===")
print("✅ O ambiente está pronto para o desenvolvimento do projeto de detecção de fraudes Pix!")